In [1]:
import cv2
import numpy as np
from collections import deque
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
new_model=tf.keras.models.load_model('digit.h5')

In [3]:
print(new_model)

In [4]:
digit_count = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6',
                    7: '7', 8: '8', 9: '9', 10: '10'}

In [5]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 28
    image_y = 28
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [ ]:
cap = cv2.VideoCapture(0)
Lower_blue = np.array([110,50,50])
Upper_blue = np.array([130,255,255])
pred_class=0
pts = deque(maxlen=512)
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)
while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(imgHSV, Lower_blue, Upper_blue)
        blur = cv2.medianBlur(mask, 15)
        blur = cv2.GaussianBlur(blur, (5, 5), 0)
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        center = None
        if len(cnts) >= 1:
            contour = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(contour) > 250:
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(img, center, 5, (0, 0, 255), -1)
                M = cv2.moments(contour)
                center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
                pts.appendleft(center)
                for i in range(1, len(pts)):
                    if pts[i - 1] is None or pts[i] is None:
                        continue
                    cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
                    cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), 5)
        elif len(cnts) == 0:
            if len(pts) != []:
                blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
                blur1 = cv2.medianBlur(blackboard_gray, 15)
                blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
                thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
                if len(blackboard_cnts) >= 1:
                    cnt = max(blackboard_cnts, key=cv2.contourArea)
                    print(cv2.contourArea(cnt))
                    if cv2.contourArea(cnt) > 2000:
                        x, y, w, h = cv2.boundingRect(cnt)
                        digit = blackboard_gray[y:y + h, x:x + w]
                        # newImage = process_letter(digit)
                        pred_probab, pred_class = keras_predict(new_model, digit)
                        print(pred_class, pred_probab)

            pts = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
        cv2.putText(img, "Conv Network :  " + str(digit_count[pred_class]), (10, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

650.0
4889.5
processed: (1, 28, 28, 1)
1 1.0
725.0
6909.0
processed: (1, 28, 28, 1)
0 1.0
2050.0
processed: (1, 28, 28, 1)
1 1.0
610.0
3826.0
processed: (1, 28, 28, 1)
7 1.0
8319.5
processed: (1, 28, 28, 1)
8 1.0
17071.5
processed: (1, 28, 28, 1)
7 1.0
1349.5
12953.0
processed: (1, 28, 28, 1)
0 1.0
5358.0
processed: (1, 28, 28, 1)
3 1.0
2819.0
processed: (1, 28, 28, 1)
1 1.0
4728.5
processed: (1, 28, 28, 1)
5 1.0
3091.5
processed: (1, 28, 28, 1)
8 1.0
1996.5
11651.5
processed: (1, 28, 28, 1)
2 1.0
1240.0
19990.0
processed: (1, 28, 28, 1)
3 1.0
7085.5
processed: (1, 28, 28, 1)
2 1.0
940.5
102.0
6295.0
processed: (1, 28, 28, 1)
1 1.0
1745.5
1442.0
22.0
165505.0
processed: (1, 28, 28, 1)
3 1.0
209.5
103.0
152.0
4175.5
processed: (1, 28, 28, 1)
2 1.0
11836.0
processed: (1, 28, 28, 1)
2 1.0
9383.0
processed: (1, 28, 28, 1)
3 1.0
2000.0
399.5
15026.0
processed: (1, 28, 28, 1)
2 1.0
124275.5
processed: (1, 28, 28, 1)
4 1.0
3673.0
processed: (1, 28, 28, 1)
7 1.0
6611.5
processed: (1, 28, 28, 1

2659.0
processed: (1, 28, 28, 1)
2 1.0
693.0
7403.0
processed: (1, 28, 28, 1)
3 1.0
2185.5
processed: (1, 28, 28, 1)
6 1.0
6141.5
processed: (1, 28, 28, 1)
0 1.0
18363.0
processed: (1, 28, 28, 1)
4 1.0
1592.0
1111.5
341.0
